In [2]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from matplotlib import pyplot as plt

In [3]:
# Define our imshow function 
def imshow(title = "Image", image = None, size = 10):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

In [ ]:
# Carrega imagens em preto e branco do diretório
file_path = "colorize/blackandwhite/"
blackandwhite_imgs = [f for f in listdir(file_path) if isfile(join(file_path, f))]
kernel = 'colorize/pts_in_hull.npy'  # Centróides de cor no espaço AB
"""
Centróides de Cor (Cluster Centers) Pontos representativos no espaço AB que definem cores possíveis. 
Usados para restringir a saída da rede.
"""

if __name__ == '__main__':
    # Carrega modelo pré-treinado (arquitetura e pesos)
    net = cv2.dnn.readNetFromCaffe("colorize/colorization_deploy_v2.prototxt",
                               "colorize/colorization_release_v2.caffemodel")

    # Configuração dos centróides como kernels de convolução 1x1
    pts_in_hull = np.load(kernel).transpose().reshape(2, 313, 1, 1)
    net.getLayer(net.getLayerId('class8_ab')).blobs = [pts_in_hull.astype(np.float32)]  # Camada de saída AB
    net.getLayer(net.getLayerId('conv8_313_rh')).blobs = [np.full([1, 313], 2.606, np.float32)]  # Bias/ajuste final

    for image in blackandwhite_imgs:
        # Pré-processamento
        img = cv2.imread(file_path+image)
        img_rgb = (img[:,:,[2, 1, 0]] * 1.0 / 255).astype(np.float32)  # Converte BGR para RGB e normaliza
        img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2LAB)  # Converte para LAB
        img_l = img_lab[:,:,0]  # Extrai canal L (luminância)

        # get original image size
        (H_orig,W_orig) = img_rgb.shape[:2] 

        # Redimensiona para entrada da rede (224x224)
        img_rs = cv2.resize(img_rgb, (224, 224))
        img_lab_rs = cv2.cvtColor(img_rs, cv2.COLOR_RGB2Lab)
        img_l_rs = img_lab_rs[:,:,0]
        img_l_rs -= 50  # Normalização: centraliza em torno de 0

        # Inferência da rede
        net.setInput(cv2.dnn.blobFromImage(img_l_rs))
        ab_dec = net.forward('class8_ab')[0,:,:,:].transpose((1,2,0))  # Predição dos canais AB

        # Pós-processamento
        ab_dec_us = cv2.resize(ab_dec, (W_orig, H_orig))  # Redimensiona AB para tamanho original
        img_lab_out = np.concatenate((img_l[:,:,np.newaxis], ab_dec_us), axis=2)  # Combina L original + AB previsto
        img_bgr_out = np.clip(cv2.cvtColor(img_lab_out, cv2.COLOR_Lab2BGR), 0, 1)  # Converte para BGR e limita valores

        # Exibe resultados
        imshow('Original', img)
        img_bgr_out = cv2.resize(img_bgr_out, (W_orig, H_orig), interpolation=cv2.INTER_AREA)  # Garante tamanho original
        imshow('Colorized', img_bgr_out)

Pontos-Chave:

- Preservação de Detalhes: O canal L original é usado na saída, mantendo a textura/brilho da imagem inicial.

- Eficiência: A rede processa imagens redimensionadas (224x224), reduzindo custo computacional.

- Interpolação: INTER_AREA é usada para redimensionar a saída, ideal para redução de tamanho (preserva detalhes).

- Normalização: Subtrair 50 no LAB (onde L varia de 0 a 100) centraliza os dados, facilitando o aprendizado da rede.